In [1]:
import os
import jax
import flax

In [2]:
import xpag
from xpag.wrappers import gym_vec_env
from xpag.buffers import DefaultEpisodicBuffer
from xpag.samplers import DefaultEpisodicSampler, HER
from xpag.goalsetters import DefaultGoalSetter
from xpag.agents import SAC, SAC_bonus
from xpag.tools import learn
from xpag.tools import mujoco_notebook_replay

gym-gfetch: 
|    gym version and path: 0.23.1 ['/Users/chenu/opt/anaconda3/envs/xpag_env/lib/python3.9/site-packages/gym']
|    REGISTERING GFetch-v0 from /Users/chenu/Desktop/PhD/github/gym-gfetch/gym_gfetch
|    REGISTERING GFetchGoal-v0 from /Users/chenu/Desktop/PhD/github/gym-gfetch/gym_gfetch
|    REGISTERING GFetchDCIL-v0 from /Users/chenu/Desktop/PhD/github/gym-gfetch/gym_gfetch


In [3]:
import gym_gfetch

In [4]:
num_envs = 1  # the number of rollouts in parallel during training
env, eval_env, env_info = gym_vec_env('GFetchDCIL-v0', num_envs)

objc[40947]: Class GLFWApplicationDelegate is implemented in both /Users/chenu/.mujoco/mujoco200/bin/libglfw.3.dylib (0x15a7ea778) and /usr/local/Cellar/glfw/3.3.2/lib/libglfw.3.3.dylib (0x15a889f10). One of the two will be used. Which one is undefined.
objc[40947]: Class GLFWWindowDelegate is implemented in both /Users/chenu/.mujoco/mujoco200/bin/libglfw.3.dylib (0x15a7ea700) and /usr/local/Cellar/glfw/3.3.2/lib/libglfw.3.3.dylib (0x15a889f38). One of the two will be used. Which one is undefined.
objc[40947]: Class GLFWContentView is implemented in both /Users/chenu/.mujoco/mujoco200/bin/libglfw.3.dylib (0x15a7ea7a0) and /usr/local/Cellar/glfw/3.3.2/lib/libglfw.3.3.dylib (0x15a889f88). One of the two will be used. Which one is undefined.
objc[40947]: Class GLFWWindow is implemented in both /Users/chenu/.mujoco/mujoco200/bin/libglfw.3.dylib (0x15a7ea818) and /usr/local/Cellar/glfw/3.3.2/lib/libglfw.3.3.dylib (0x15a88a000). One of the two will be used. Which one is undefined.


name:  obj0
filename :
 /Users/chenu/Desktop/PhD/github/dcil/demos/fetchenv/demo_set/1.demo
self.L_budgets =  [10, 10, 14, 14, 12, 18, 20, 14, 26, 16, 16, 14, 40, 2]
name:  obj0
filename :
 /Users/chenu/Desktop/PhD/github/dcil/demos/fetchenv/demo_set/1.demo
self.L_budgets =  [10, 10, 14, 14, 12, 18, 20, 14, 26, 16, 16, 14, 40, 2]


objc[40965]: Class GLFWApplicationDelegate is implemented in both /Users/chenu/.mujoco/mujoco200/bin/libglfw.3.dylib (0x151cad778) and /usr/local/Cellar/glfw/3.3.2/lib/libglfw.3.3.dylib (0x151d4cf10). One of the two will be used. Which one is undefined.
objc[40965]: Class GLFWWindowDelegate is implemented in both /Users/chenu/.mujoco/mujoco200/bin/libglfw.3.dylib (0x151cad700) and /usr/local/Cellar/glfw/3.3.2/lib/libglfw.3.3.dylib (0x151d4cf38). One of the two will be used. Which one is undefined.
objc[40965]: Class GLFWContentView is implemented in both /Users/chenu/.mujoco/mujoco200/bin/libglfw.3.dylib (0x151cad7a0) and /usr/local/Cellar/glfw/3.3.2/lib/libglfw.3.3.dylib (0x151d4cf88). One of the two will be used. Which one is undefined.
objc[40965]: Class GLFWWindow is implemented in both /Users/chenu/.mujoco/mujoco200/bin/libglfw.3.dylib (0x151cad818) and /usr/local/Cellar/glfw/3.3.2/lib/libglfw.3.3.dylib (0x151d4d000). One of the two will be used. Which one is undefined.


name:  obj0
filename :
 /Users/chenu/Desktop/PhD/github/dcil/demos/fetchenv/demo_set/1.demo
self.L_budgets =  [10, 10, 14, 14, 12, 18, 20, 14, 26, 16, 16, 14, 40, 2]
name:  obj0
filename :
 /Users/chenu/Desktop/PhD/github/dcil/demos/fetchenv/demo_set/1.demo
self.L_budgets =  [10, 10, 14, 14, 12, 18, 20, 14, 26, 16, 16, 14, 40, 2]


objc[40974]: Class GLFWApplicationDelegate is implemented in both /Users/chenu/.mujoco/mujoco200/bin/libglfw.3.dylib (0x1584c7778) and /usr/local/Cellar/glfw/3.3.2/lib/libglfw.3.3.dylib (0x158566f10). One of the two will be used. Which one is undefined.
objc[40974]: Class GLFWWindowDelegate is implemented in both /Users/chenu/.mujoco/mujoco200/bin/libglfw.3.dylib (0x1584c7700) and /usr/local/Cellar/glfw/3.3.2/lib/libglfw.3.3.dylib (0x158566f38). One of the two will be used. Which one is undefined.
objc[40974]: Class GLFWContentView is implemented in both /Users/chenu/.mujoco/mujoco200/bin/libglfw.3.dylib (0x1584c77a0) and /usr/local/Cellar/glfw/3.3.2/lib/libglfw.3.3.dylib (0x158566f88). One of the two will be used. Which one is undefined.
objc[40974]: Class GLFWWindow is implemented in both /Users/chenu/.mujoco/mujoco200/bin/libglfw.3.dylib (0x1584c7818) and /usr/local/Cellar/glfw/3.3.2/lib/libglfw.3.3.dylib (0x158567000). One of the two will be used. Which one is undefined.


In [5]:
agent = SAC(
    env_info['observation_dim'] if not env_info['is_goalenv']
    else env_info['observation_dim'] + env_info['desired_goal_dim'],
    env_info['action_dim'],
    {}
)
sampler = DefaultEpisodicSampler() if not env_info['is_goalenv'] else HER(env.env_fns[0]().compute_reward)
print("sampler = ", sampler)

buffer = DefaultEpisodicBuffer(
    max_episode_steps=env_info['max_episode_steps'],
    buffer_size=1_000_000,
    sampler=sampler
)
goalsetter = DefaultGoalSetter()

name:  obj0
filename :
 /Users/chenu/Desktop/PhD/github/dcil/demos/fetchenv/demo_set/1.demo
self.L_budgets =  [10, 10, 14, 14, 12, 18, 20, 14, 26, 16, 16, 14, 40, 2]
sampler =  <xpag.samplers.HER.HER object at 0x7fdb16dafac0>


In [6]:
batch_size = 256
gd_steps_per_step = 1
start_training_after_x_steps = env_info['max_episode_steps'] * 10
max_steps = 50_000
evaluate_every_x_steps = 1_000
save_agent_every_x_steps = 100_000
save_dir = os.path.join(os.path.expanduser('~'), 'results', 'xpag', 'train_mujoco')
save_episode = True
plot_projection = None

In [7]:
import os
import numpy as np
import matplotlib.pyplot as plt
from xpag.tools.eval import single_rollout_eval
from xpag.tools.utils import hstack
from xpag.tools.logging import eval_log_reset
from xpag.tools.timing import timing_reset
from xpag.buffers import Buffer
from xpag.agents.agent import Agent
from xpag.goalsetters.goalsetter import GoalSetter
from typing import Dict, Any, Union
from collections import OrderedDict

In [8]:
def plot_traj(trajs, eval_env, save_dir, it=0):
    fig = plt.figure()
    fig.set_figheight(5.2)
    fig.set_figwidth(5.2)
    ax = fig.add_subplot(projection='3d')
    
    ax.set_xlim((0., 1.5))
    ax.set_ylim((-1., 2.))
    ax.set_zlim((0., 2.2))
    
    fake_env = eval_env.env_fns[0]().unwrapped
    fake_env.reset()
    
    #print("traj = ", traj)
    for traj in trajs:
        
        for i in range(traj[0].shape[0]):
            success = False
            X = []
            Y = []
            Z = []
            #print("\n desired_goal = ", fake_env.goal)
            for goal in traj:
                #print("achieved_goal = ", goal)
                #if np.linalg.norm(goal[i] - fake_env.goal) < 0.05:
                    #print("achieved_goal[i] = ", goal[0])
                    #success = True
                X.append(goal[i][0])
                Y.append(goal[i][1])
                Z.append(goal[i][2])
            if success:
                ax.plot(X, Y, Z, c="m")
            else:
                ax.plot(X, Y, Z, c="pink")

    #X_eval = [goal[0] for goal in eval_traj]
    #Y_eval = [goal[1] for goal in eval_traj]
    #Z_eval = [goal[2] for goal in eval_traj]
    #ax.plot(X_eval, Y_eval, Z_eval, c = "red")

    visu_success_zones(eval_env, ax)
    
    for azim_ in range(45,360,90):
        #ax.view_init(elev=0., azim = azim_)
        ax.view_init(azim = azim_)
        plt.savefig(save_dir + "/trajs_" + str(azim_) + "_it_" + str(it) + ".png")
    
    plt.close(fig)
    return

def visu_success_zones(eval_env, ax):
    """
    Visualize success zones as sphere of radius eps_success around skill-goals
    """
    fake_env = eval_env.env_fns[0]()
    L_states = fake_env.skill_manager.L_states
    i = 0
    for state in L_states:
        goal = fake_env.project_to_goal_space(state)

        u, v = np.mgrid[0:2*np.pi:20j, 0:np.pi:10j]

        x = goal[0] + 0.075*np.cos(u)*np.sin(v)
        y = goal[1] + 0.075*np.sin(u)*np.sin(v)
        z = goal[2] + 0.075*np.cos(v)
        if i ==0:
            ax.plot_wireframe(x, y, z, color="red", alpha = 0.1)
        else:
            ax.plot_wireframe(x, y, z, color="blue", alpha = 0.1)
        i+= 1

    return

In [9]:
env

<ResetDoneVecWrapper<AsyncVectorEnv instance>>

In [10]:
eval_log_reset()
timing_reset()
observation = env.reset()
print("achieved_goal = ", observation["achieved_goal"])
trajs = []
traj = []
num_rollouts = 0
num_success = 0

for i in range(max_steps // env_info["num_envs"]):
    traj.append(observation["achieved_goal"])
    if not i % max(evaluate_every_x_steps // env_info["num_envs"], 1):
        single_rollout_eval(
            i * env_info["num_envs"],
            eval_env,
            env_info,
            agent,
            save_dir=save_dir,
            plot_projection=plot_projection,
            save_episode=save_episode,
        )
        plot_traj(trajs, eval_env, save_dir, it=i)
        traj = []
        trajs = []
        
        if num_rollouts > 0:
            print("ratio is: ", float(num_success/num_rollouts))
            num_rollouts = 0
            num_success = 0

    if not i % max(save_agent_every_x_steps // env_info["num_envs"], 1):
        if save_dir is not None:
            agent.save(os.path.join(save_dir, "agent"))

    if i * env_info["num_envs"] < start_training_after_x_steps:
        action = env_info["action_space"].sample()
    else:
        action = agent.select_action(
            observation
            if not env_info["is_goalenv"]
            else hstack(observation["observation"], observation["desired_goal"]),
            deterministic=False,
        )
        for _ in range(max(round(gd_steps_per_step * env_info["num_envs"]), 1)):
            batch = buffer.sample(batch_size)
            #print("batch reward = ", batch["reward"][:10])
            #print("\nbatch.done = ", batch["done"][:5])
            #print("batch.truncation = ", batch["truncation"][:5])
            #print("batch.info = ", batch["info"])
            info_train = agent.train_on_batch(batch)
            #print("\nis_success = ", info_train["is_success"])
            #print("is_not_relabelled = ", info_train["is_not_relabelled"])
            #print("next_goal_avail = ", info_train["next_goal_avail"])
            #print("add_bonus = ", info_train["add_bonus"])
            #print("rewards = ", max(info_train["rewards"]))
            #print("\nmasks = ", info_train["masks"])
            #print("rewards = ", info_train["rewards"])
            #print("target_q = ", info_train["target_q"])
            #print("next_q = ", info_train["next_q"])

    next_observation, reward, done, info = env.step(action)
    #print("reward = ", reward)
    #print("achieved_goal = ", observation["achieved_goal"])
    #print("reward.shape = ", reward.shape)
    #print("action = ", action.shape)
    #print("done = ", done)
    #print("info = ", info)
    
    _info = {}
    for key in info[0].keys():
        _info[key] = np.array([info[i][key] for i in range(len(info))]).reshape(env_info["num_envs"],-1)
    #print("_info = ", _info)
    
    step = {
        "observation": observation,
        "action": action.reshape(env_info["num_envs"],-1),
        "reward": reward.reshape(env_info["num_envs"],-1),
        "truncation": _info["truncation"],
        "done": done.reshape(env_info["num_envs"],-1),
        "next_observation": next_observation,
    }
    #print("buffer.current_idxs = ", buffer.current_idxs)
    #print("buffer.current_t = ", buffer.current_t)
    #print("step['observation'] = ", step["observation"])
    
    if env_info["is_goalenv"]:
        step["is_success"] = _info["is_success"]
    buffer.insert(step)
    observation = next_observation

    if done.max():
        num_rollouts += 1
        if _info["is_success"].max() == 1:
            num_success += 1
        traj.append(observation["achieved_goal"])
        trajs.append(traj)
        traj = []
        # use store_done() if the buffer is an episodic buffer
        if hasattr(buffer, "store_done"):
            buffer.store_done()
        observation = env.reset_done()
        #print("achieved_goal = ", observation["achieved_goal"])

achieved_goal =  [[0.31544292 1.39724488 0.8232876 ]]


[           0 steps] [training time (ms) += 0         ] [ep reward: 0.000          ] [success: 0.00]


Logging in /Users/chenu/results/xpag/train_mujoco
name:  obj0
filename :
 /Users/chenu/Desktop/PhD/github/dcil/demos/fetchenv/demo_set/1.demo
self.L_budgets =  [10, 10, 14, 14, 12, 18, 20, 14, 26, 16, 16, 14, 40, 2]
name:  obj0
filename :
 /Users/chenu/Desktop/PhD/github/dcil/demos/fetchenv/demo_set/1.demo
self.L_budgets =  [10, 10, 14, 14, 12, 18, 20, 14, 26, 16, 16, 14, 40, 2]


[        1000 steps] [training time (ms) += 16244     ] [ep reward: 0.000          ] [success: 0.00]


name:  obj0
filename :
 /Users/chenu/Desktop/PhD/github/dcil/demos/fetchenv/demo_set/1.demo
self.L_budgets =  [10, 10, 14, 14, 12, 18, 20, 14, 26, 16, 16, 14, 40, 2]
name:  obj0
filename :
 /Users/chenu/Desktop/PhD/github/dcil/demos/fetchenv/demo_set/1.demo
self.L_budgets =  [10, 10, 14, 14, 12, 18, 20, 14, 26, 16, 16, 14, 40, 2]
ratio is:  0.03508771929824561


...

name:  obj0
filename :
 /Users/chenu/Desktop/PhD/github/dcil/demos/fetchenv/demo_set/1.demo
self.L_budgets =  [10, 10, 14, 14, 12, 18, 20, 14, 26, 16, 16, 14, 40, 2]
name:  obj0
filename :
 /Users/chenu/Desktop/PhD/github/dcil/demos/fetchenv/demo_set/1.demo
self.L_budgets =  [10, 10, 14, 14, 12, 18, 20, 14, 26, 16, 16, 14, 40, 2]
ratio is:  0.16417910447761194


[       48000 steps] [training time (ms) += 18800     ] [ep reward: 0.000          ] [success: 0.00]


name:  obj0
filename :
 /Users/chenu/Desktop/PhD/github/dcil/demos/fetchenv/demo_set/1.demo
self.L_budgets =  [10, 10, 14, 14, 12, 18, 20, 14, 26, 16, 16, 14, 40, 2]
name:  obj0
filename :
 /Users/chenu/Desktop/PhD/github/dcil/demos/fetchenv/demo_set/1.demo
self.L_budgets =  [10, 10, 14, 14, 12, 18, 20, 14, 26, 16, 16, 14, 40, 2]
ratio is:  0.22058823529411764


[       49000 steps] [training time (ms) += 19099     ] [ep reward: 0.000          ] [success: 0.00]


name:  obj0
filename :
 /Users/chenu/Desktop/PhD/github/dcil/demos/fetchenv/demo_set/1.demo
self.L_budgets =  [10, 10, 14, 14, 12, 18, 20, 14, 26, 16, 16, 14, 40, 2]
name:  obj0
filename :
 /Users/chenu/Desktop/PhD/github/dcil/demos/fetchenv/demo_set/1.demo
self.L_budgets =  [10, 10, 14, 14, 12, 18, 20, 14, 26, 16, 16, 14, 40, 2]
ratio is:  0.17391304347826086
